# Assignment 1 FIT5145 - Introduction to Data Science

Nhung Seidensticker

Monash ID: 29395968

User Documentation

This assignment has 2 tasks. 

Task 1: Unemployment Rate and House Prices

Task 2: Choice


# Task 1: Unemployment Rate and House Prices

In this task, you are required to visualise the relationship between:
 * unemployment rate
 * house price index
 * population of different Australian state
 * gain insights from how these relations and trends change over time using motion charts

(A) Generate motion chart showing: 
   * Unemployment rate on the x-axis
   * House price index on the y-axis
   * Colour bubbles by state
   * Size of bubbles by population
> if the **time** on the motion chart is not displaying correcting - covert field into a simply string. 
> import datetime
> data.Time = data.Time.apply(lambda x: datetime.datetime.strtime(x,'%d/%m/%y'))

(B) Answer the following questions:
    1. What are the overall patterns across different state in Australia?
    2. Is there any relationship between unemployment rate and the house price index?
    3. Which state has high unemployment rate and high house price index
        (you may point out different period of time) 
        * how about low unemployment rate and low house price index?
    4. What is the performance of the most populated states like NSW and VIC?
    



### Data files

Data files provided are originally downloaded from:
   * The Department of Employment
   * Australian Bureau of Statistics (ABS)

They are: 

* **ERP_by state and gender.csv**: Quarterly data about estimated resident population, between 1/12/2005 and 1/6/2015
* **House Price Index**: Quarterly data of the Residential Property Price Index of the capital cities, between 1/12/2005 and 1/6/2015
* **SA4 Time Series - October 2016.xls**: Monthly data of employment and unemployment rates, between 1/2/1978 and 1/6/2015


## Importing relevant packages

Relevant packages are being imported to assist in the creation of motion chart. 

***%matplotlib inline*** is a magic functon to ensure that output of plotting commands is displayed inline with Jupyter notebook, directly below the code cell that produced it. The resulting plot will then also be stored in the notebook itself. 

In [338]:
import pandas as pd
from motionchart.motionchart import MotionChart
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from scipy.stats import linregress
import datetime
#import calendar
from functools import reduce

%matplotlib inline

## Importing Data

### Explore the data
Importing the relevant datasets using pandas

In [339]:
House_price = pd.read_csv("./02_In/House Price Index.csv")

SA4_Time = pd.ExcelFile("./02_In/SA4 Time Series - October 2016.xls")
#this tells pandas which sheet to read
SA4_df = SA4_Time.parse("Time Series")

ERP_Population = pd.ExcelFile("./02_In/ERP_Population.xls")



## ERP_by state and gender

### Wrangling

The data needs to be refined to get rid of superfluous rows and columns.  
1. Remove the first 8 rows of the data
2. Return fields to persons only, as this is what we are interested in
3. Rename the fields to something more sensible
4. Format the date field using pandas to_datetime
5. Change the shape of the dataframe by using melt. 
    *for motion chart need a field with State names and melt will do this.* 
6. Extract data from the relevant date range only


In [340]:
#this tells pandas which sheet to read
ERP_Population_df = ERP_Population.parse("Data1")

#drop the first 8 rows as the data is irrelevant
ERP_Population_df.drop(ERP_Population_df.index[:9], inplace = True)

#Create a dataframe with the relevant columns only
ERP_Population_df = ERP_Population_df.iloc[:, [18,19,20,21,22,23,24,25]]

ERP_Population_df.reset_index(level = 0, inplace = True)
#Rename the columns so that they are consistent with the other data
ERP_Population_df = ERP_Population_df.rename(columns={'index': 'Date',
                                      'Estimated Resident Population ;  Persons ;  New South Wales ;':'NSW',
                                     'Estimated Resident Population ;  Persons ;  Victoria ;':'VIC',
                                     'Estimated Resident Population ;  Persons ;  Queensland ;':'QLD',
                                     'Estimated Resident Population ;  Persons ;  South Australia ;':'SA',
                                     'Estimated Resident Population ;  Persons ;  Western Australia ;':'WA',
                                     'Estimated Resident Population ;  Persons ;  Tasmania ;':'TAS',
                                     'Estimated Resident Population ;  Persons ;  Northern Territory ;':'NT',
                                     'Estimated Resident Population ;  Persons ;  Australian Capital Territory ;':'ACT',
                                     'Estimated Resident Population ;  Persons ;  Australia ;':'Australia',})

# #Ensure the date is actually a date
ERP_Population_df['Date'] = pd.to_datetime(ERP_Population_df.Date)
ERP_Population_df.head()
#change the data so that the State and Population values are in separate columns, and rename the columns
ERP_temp = pd.melt(ERP_Population_df, id_vars = 'Date', value_vars = ['NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT'])
erp_population= ERP_temp.rename(columns = {'variable': 'State/Territory','value': 'Population'})

#specify the date range to read the data
erp_population = erp_population[(erp_population['Date'] >= '2005-12-01') & (erp_population['Date'] <= '2015-06-1')]
#determine the months to get the data frome
months = [12,3,6,9]
#grab the data from that month only
t1_population_df = erp_population[erp_population['Date'].map(lambda t: t.month in months)]


## House Price Index

### Step 1: Look at the data
Looking at the data to see if it is characterised by city. 

#### Definition of House Price Index
The House Price Index (HPI) measures price change for established detached houses within eight capital cities in Australia over time. 
Established houses are defined as detached residential dwellings on their own block of land, including new houses sold as a house/land package. 

1. Bring in the data
2. Rename the columns names
3. Format the date field using pandas to_datetime

In [341]:
#Rename the columns so that they are consistent
house_price_df = House_price.rename(columns={'Unnamed: 0': 'Date',
                                             'Residential Property Price Index ;  Sydney ;': 'NSW',
                                            'Residential Property Price Index ;  Melbourne ;': 'VIC',
                                            'Residential Property Price Index ;  Brisbane ;': 'QLD',
                                            'Residential Property Price Index ;  Adelaide ;': 'SA',
                                            'Residential Property Price Index ;  Perth ;': 'WA',
                                            'Residential Property Price Index ;  Hobart ;': 'TAS',
                                            'Residential Property Price Index ;  Darwin ;': 'NT',
                                            'Residential Property Price Index ;  Canberra ;': 'ACT'})

#Ensure that the Date field is actually date
house_price_df['Date'] = pd.to_datetime(house_price_df.Date)

#Transform the data so that the State and house price index are in columns with the right titles. 
house_price_temp = pd.melt(house_price_df, id_vars = 'Date', value_vars = ['NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT'])
HPI = house_price_temp.rename(columns = {'variable': 'State/Territory','value': 'HPI'})

## SA4 Time Series - October 2016

### Step 1: Look at the data
The data is captured monthly, therefore will only extract the quarterly to align with the other datasets.  

1. Bring in the data
2. Rename the columns
3. Format the date field using: 

In [342]:
#Ensure that the Date field is actually date
SA4_df['Date'] = pd.to_datetime(SA4_df.Date)

#Select that data at only at the specific date range as the other data
SA4_df = SA4_df[(SA4_df['Date'] >= '2005-12-01') & (SA4_df['Date'] <= '2015-06-1')]
SA4_df['Date'] - pd.offsets.MonthBegin(1, normalize=True)
months = [12, 3, 6, 9]
employment_df = SA4_df[SA4_df["Date"].map(lambda t: t.month in months)]

#Take only the relevant columns into a dataframe
unemployment = employment_df.iloc[:, [0,1] + [3]]


## Combine the data

1. Merge the data using reduce and merge
2. Change the date format so that it displays in a sensible way within the motion chart

In [343]:
dfs = [t1_population_df, HPI, unemployment]
#merge the data frames into 1. Reduce is used as it merges the first two, then the third
unemployment_hpi = reduce(lambda left, right: pd.merge(left, right, on = ['Date','State/Territory']),dfs)
#Change the date time string so that it looks sensible in the chart
unemployment_hpi.Date=pd.to_datetime(unemployment_hpi.Date).dt.strftime('%d-%b-%y')

#Code below is to output the combined dataframe into a *.csv. 
#umemployment_hpi.to_csv('umemployment rate and HPI.csv')

## Finding min and max
To understand the data, find the min and max for the category of HPI and unemployment

In [344]:
max_hpi = unemployment_hpi.nlargest(3, 'HPI')
max_unemployment = unemployment_hpi.nlargest(3, 'Unemployment Rate (15+)')

min_hpi = unemployment_hpi.nsmallest(3, 'HPI')
min_unemployment = unemployment_hpi.nsmallest(3, 'Unemployment Rate (15+)')

#max_hpi
#max_unemployment
#min_hpi
#min_unemployment

## Generate Motion Chart

The following html code block is just to make sure that you will see the entire motion chart nicely in the output cell. Although the motion chart is not being displayed in the jupyterHub notebook, I've kept this here just in case it is fixed. 



In [345]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

The following code is putting the data into a motion chart. The use of to_browser() instead of to_notebook() is used, due to the technical difficulties in Monash's JupyterHub not displaying the data. 

In [346]:
unemployment_hpi_plot = MotionChart(umemployment_hpi, key='Date', x='Unemployment Rate (15+)', y='HPI', xscale='linear', yscale='linear',
                     size='Population', color='State/Territory', category='State/Territory')

unemployment_hpi_plot.to_notebook()

# Task 2: Family Structure and Residential Property Price Index

The aim of this task is to supplement the first task to determine if there is a change in the family structure of 'Family with Children' due to pressures in the Residential Property Price Index. 

## Data

Data obatained from http://stat.data.abs.gov.au/

**Data used**
1. Projected_Families.csv
2. Household_projections.csv
3. ERP_Populations.xls - taken from task 1
4. RPPI.xls


### Import Data

In [211]:
household_projections = pd.read_csv("./02_In/HouseHoldProjection.csv")
projected_families = pd.read_csv("./02_In/Projected_families.csv")
residential_prop_price = pd.ExcelFile("./02_In/RPPI.xls")


## Household Projections

### Step 1: Look at the data
Looking at the data to see if it is characterised by city. 

#### Wrangling the data

1. Consolidate the data to one State by renaming values
2. Reformat the data frame 


In [337]:
#Combine the data so that they are combined by state
household_projections["Region"].replace(["Greater Brisbane", "Brisbane", "Rest of Qld", 'Queensland'], "QLD", inplace=True)
household_projections["Region"].replace(["Greater Darwin", "Northern Territory", "Rest of NT", "Darwin"], "NT", inplace=True)
household_projections["Region"].replace(["Greater Melbourne", "Melbourne", "Rest of Vic.", 'Melbourne', 'Victoria'], "VIC", inplace=True)
household_projections["Region"].replace(["Greater Perth", "Perth", "Rest of WA", 'Western Australia'], "WA", inplace=True)
household_projections["Region"].replace(["Greater Adelaide", "Adelaide", "Rest of SA", 'South Australia'], "SA", inplace=True)
household_projections["Region"].replace(["Greater Sydney","Rest of NSW", 'Sydney', "New South Wales"], "NSW", inplace=True)
household_projections["Region"].replace(["Australian Capital Territory"], "ACT", inplace=True)
household_projections["Region"].replace(["Greater Hobart", "Hobart", "Rest of Tas.", 'Tasmania'], "TAS", inplace=True)

#create a dataframe with the columns needed
hh_projections = household_projections[['Time','Region', 'Household Type', 'Value']]
#transform the dataframe using pivot to have the right columns and rows
p_hh_projections = hh_projections.pivot_table(index=['Time', 'Region'], columns='Household Type', values='Value',aggfunc='sum')

#turn the pivot table into a working dataframe
projected_households = pd.DataFrame(p_hh_projections.to_records())

#find percentage of family households
projected_households['Percentage Family Households'] = (projected_households['Family households'] / projected_households['Total households']) *100

#refine the dataframe to only use the relevant fields
total_households_proj = projected_households[['Time', 'Region', 'Percentage Family Households']]


,Time,Region,Percentage Family Households
0,2011,ACT,70.257852
1,2011,Australia,71.252541
2,2011,NSW,71.687344
3,2011,NT,73.152692
4,2011,QLD,72.050011


## Projected families

### Wrangling the data

1. Consolidate the data to one State by renaming values
2. Reformat the data frame 

In [329]:
#Combine the data so that they are combined by state
projected_families["Region"].replace(["Greater Brisbane", "Brisbane", "Rest of Qld", 'Queensland'], "QLD", inplace=True)
projected_families["Region"].replace(["Greater Darwin", "Northern Territory", "Rest of NT", "Darwin"], "NT", inplace=True)
projected_families["Region"].replace(["Greater Melbourne", "Melbourne", "Rest of Vic.", 'Melbourne', 'Victoria'], "VIC", inplace=True)
projected_families["Region"].replace(["Greater Perth", "Perth", "Rest of WA", 'Western Australia'], "WA", inplace=True)
projected_families["Region"].replace(["Greater Adelaide", "Adelaide", "Rest of SA", 'South Australia'], "SA", inplace=True)
projected_families["Region"].replace(["Greater Sydney","Rest of NSW", 'Sydney', "New South Wales"], "NSW", inplace=True)
projected_families["Region"].replace(["Australian Capital Territory"], "ACT", inplace=True)
projected_families["Region"].replace(["Greater Hobart", "Hobart", "Rest of Tas.", 'Tasmania'], "TAS", inplace=True)

#create a dataframe with the columns needed
family_types = projected_families[['Time', 'Region', 'Family Type','Value']]
#transform the dataframe using pivot to have the right columns and rows
pivot_familty_types = family_types.pivot_table(index = ['Time', 'Region'], columns = 'Family Type', values = 'Value', aggfunc = 'sum')

#turn the pivot table into a working dataframe
proj_family_types = pd.DataFrame(pivot_familty_types.to_records())

#find percentage of family households
proj_family_types['Percentage couples with children'] = (proj_family_types['Couple families with children'] / proj_family_types['Total families']) *100

#refine the dataframe to only use the relevant fields
total_families_proj = proj_family_types[['Time', 'Region', 'Percentage couples with children']]
total_families_proj.head()

,Time,Region,Percentage couples with children
0,2011,ACT,45.693782
1,2011,Australia,44.144451
2,2011,NSW,45.101885
3,2011,NT,45.973897
4,2011,QLD,42.376759


## ERP_by state and gender

### Wrangling
The data is being reused from the first task

The data is changed to fit the date range for task 2. 

In [330]:
#specify the date range to read the data
erp_population = erp_population[(erp_population['Date'] >= '2005-12-01') & (erp_population['Date'] <= '2017-12-1')]
#determine the months to get the data frome
month = [12]
#grab the data from that month only
t2_population_df = erp_population[erp_population['Date'].map(lambda t: t.month in month)]

#Change the column names so that they are consistent
t2_population = t2_population_df.rename(columns={'State/Territory': 'Region', 'Date': 'Time'})

#only grab the year 
t2_population['Time'] = t2_population_df['Date'].dt.year

## Residential Property Price Index (RPPI)

### Wrangling

The data needs to be refined to get rid of superfluous rows and columns.  
1. Remove the first 8 rows of the data
2. Return fields to persons only, as this is what we are interested in
3. Rename the fields to something more sensible
4. Format the date field using pandas to_datetime
5. Change the shape of the dataframe by using melt. 
    *for motion chart need a field with State names and melt will do this.* 
6. Extract data from the relevant date range only


In [331]:
#this tells pandas which sheet to read
rppi_df = residential_prop_price.parse("Data1")

#drop the first 8 rows as the data is irrelevant
rppi_df.drop(rppi_df.index[:9], inplace = True)

#Create a dataframe with the relevant columns only
rppi_df = rppi_df.iloc[:, 0:8]
rppi_df

rppi_df.reset_index(level = 0, inplace = True)
#Rename the columns so that they are consistent with the other data
rppi_df = rppi_df.rename(columns={'index': 'Time',
                                      'Residential Property Price Index ;  Sydney ;':'NSW',
                                     'Residential Property Price Index ;  Melbourne ;':'VIC',
                                     'Residential Property Price Index ;  Brisbane ;':'QLD',
                                     'Residential Property Price Index ;  Adelaide ;':'SA',
                                     'Residential Property Price Index ;  Perth ;':'WA',
                                     'Residential Property Price Index ;  Hobart ;':'TAS',
                                     'Residential Property Price Index ;  Darwin ;':'NT',
                                     'Residential Property Price Index ;  Canberra ;':'ACT'})


#change the data so that the State and Population values are in separate columns, and rename the columns
rppi_temp = pd.melt(rppi_df, id_vars = 'Time', value_vars = ['NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT'])
rppi_by_state= rppi_temp.rename(columns = {'variable': 'Region','value': 'RPPI'})

#Ensure the date is actually a date
rppi_df['Time'] = pd.to_datetime(ERP_Population_df.Date)

#specify the date range to read the data
rppi_by_state = rppi_by_state[(rppi_by_state['Time'] >= '2005-12-01') & (rppi_by_state['Time'] <= '2017-12-1')]

#determine the months to get the data frome
months = [12]
#grab the data from that month only
rppi_df_chart = rppi_by_state[rppi_by_state['Time'].map(lambda t: t.month in months)]

#copy the dataframe so that the year can be extracted
rppi_to_chart = rppi_df_chart.filter(['Time', 'Region', 'RPPI'], axis = 1)

#only grab the year 
rppi_to_chart['Time'] = rppi_df_chart['Time'].dt.year


# Combine the data

1. Merge the data using reduce and merge

In [336]:
dataframes = [total_households_proj, total_families_proj, t2_population,rppi_to_chart]

#Consolidate the dataframes. reduce is used to merge 3 datframes at once. 
households_families_rppi = reduce(lambda left, right: pd.merge(left, right, on = ['Time','Region']),dataframes) 
households_families_rppi.head()

#Code below is to output the combined dataframe into a *.csv. 
#households_families_rppi.to_csv('family structure and RPPI.csv')

# Generate Motion Chart

The following html code block is just to make sure that you will see the entire motion chart nicely in the output cell. Although the motion chart is not being displayed in the jupyterHub notebook, I've kept this here just in case it is fixed. 

In [323]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [335]:
household_family_plot = MotionChart(households_families_rppi, key='Time', x='Percentage couples with children', y='RPPI', xscale='linear', yscale='linear',
                     size='Population', color='Region', category='Region')

household_family_plot.to_notebook()